# Real-time Visualization Widget


REBOUND comes with built-in real-time 3D visualizations. This feature can be used without python. This notebookbook describes how you can access real time visualizations directly from a Jupyter notebooks. Under the hood, the code that is running is the same as the one providing OpenGL visualizations. 

Using the real-time visualization widget makes setting up a simulation and debugging it very interactive and intuitive. You immediately see if the particles are roughly in the place where you want them, doing roughly what you expect them to do.

For this widget to work, you will need a browser that supports WebAssembly and WebGL. All modern browsers should have those features enabled by default.

Let us start this demo by setting up an empty simulation and calling the `widget()` function on the simulation object. This will create a new widget and attach it to the simulation.

In [1]:
import rebound
sim = rebound.Simulation()
sim.widget(size=(400,400))

Next up, lets add some particles to the simulation. The widget updates automatically when a particle gets added or removed. 

In [2]:
sim.add(m=1) # add a star

In [3]:
for i in range(10):
    sim.add(m=1e-3,a=0.4+0.1*i,inc=0.03*i,omega=5.*i) # Jupiter mass planets on close orbits
sim.move_to_com() # Move to the center of mass frame

Drag the widget with your mouse or touchpad to look at the simulation from different angles. Keep the shift key pressed while you drag to zoom in or out. Try pressing the "w" button. This will toggle orbits on and off.

Next, we will try to integrate the orbits forward in time. Because the planets are very massive and on close to each other, the system will go unstable very quickly. By default, REBOUND is using the IAS15 integrator which can resolve close encounter. During each close encounter the instantaneous orbits of the planets show in the widget will change rapidly.  

In [4]:
sim.integrate(500)

The widget will remain open until you delete the simulation.

In [5]:
del sim

There are some important things to understand regarding how these widgets work. We'll go over those next.

### REBOUND is a web server!
REBOUND includes its own web server. The widget connects to this web server to get the visualization code (a version of REBOUND compiled to WebAssembly) and the simulation data itself (in the form of Simulationarchive binary data). 
By default, the port the web server uses is 1234. You can start this webserver manually (without the widget) by running

In [6]:
sim = rebound.Simulation()
sim.start_server()

You can then connect to the REBOUND webserver by opening a new browser window at http://localhost:1234 or http://127.0.0.1:1234. If you want to stop the server, but not delete the simulation, you can run:

In [7]:
sim.stop_server()

### Multiple simulations

You can visualize multiple simulations at the same time. Each simulation needs to use have its port. For example:

In [8]:
sim1 = rebound.Simulation()
sim1.start_server(port=1234)
sim2 = rebound.Simulation()
sim2.widget(port=1235, size=(200,200))

Simulation 1 is shown in the widget above. You can view simulation 2 by going to http://localhost:1235 or by opening another widget:

In [10]:
sim2.widget(size=(200,200))

In [11]:
del sim1, sim2

### Visualizing simulations on a remote server

Sometimes you might run a simulation on a remote server or computing cluster. When connecting to the remote server using ssh, you can forward the visualization ports the same way you would forward the port required for jupyter notebooks (8888 by default). REBOUND uses port 1234 by default, so you might want to enable port forwarding using 

```bash
ssh username@remotecomputer -L 1234:localhost:1234
```

You can then connect to the visualization as usual, either using the widget you get with `sim.widget()` or by pointing your browser to http://localhost:1234.

### Security and resource considerations

The REBOUND web server provides a quick and easy way to visualize simulations. It is not intended to be exposed to the public internet because a malicious person might be able to gain access to your computer. 

The visualization uses a considerable amount of CPU resources. You might want to disable it if you no longer use it. 

Depending on your simulation (e.g. for simulations with a large number of particles), the communication between the REBOUND web server and your browser might use a lot of bandwidth. 